In [1]:
%load_ext autoreload
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn import metrics
from datetime import datetime
from src.average_precision import calculate_score
import numpy as np
import pickle
from keras.preprocessing import sequence
%autoreload 2
from src.average_precision import calculate_delta
import pandas as pd

Using TensorFlow backend.


In [35]:
X_train = np.array(pickle.load(open("data/rnn_datasets/X_train.pickle", "rb")))
y_train = np.array(pickle.load(open("data/rnn_datasets/y_train.pickle", "rb")))

In [36]:
X_train, X_test, y_train, y_test = train_test_split(
                                    X_train, y_train, test_size=0.20, random_state=42)

In [37]:
X_train = sequence.pad_sequences(X_train, maxlen=5)
X_test = sequence.pad_sequences(X_test, maxlen=5)

In [39]:
model = Sequential()
model.add(LSTM(256, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1], activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam")

In [41]:
model.fit(X_train, y_train, nb_epoch=10, batch_size=512)

Epoch 1/10
750734/750734 [==============================] - 16s - loss: 1.2818    
Epoch 2/10
750734/750734 [==============================] - 16s - loss: 1.2268    
Epoch 3/10
750734/750734 [==============================] - 17s - loss: 1.2108    
Epoch 4/10
750734/750734 [==============================] - 17s - loss: 1.2031    
Epoch 5/10
750734/750734 [==============================] - 18s - loss: 1.1988    
Epoch 6/10
750734/750734 [==============================] - 18s - loss: 1.1963    
Epoch 7/10
750734/750734 [==============================] - 18s - loss: 1.1946    
Epoch 8/10
750734/750734 [==============================] - 18s - loss: 1.1933    
Epoch 9/10
750734/750734 [==============================] - 18s - loss: 1.1924    
Epoch 10/10
750734/750734 [==============================] - 18s - loss: 1.1916    


In [42]:
preds = np.round(model.predict_proba(X_test), 4)

187616/187684 [============================>.] - ETA: 0s

In [43]:
score = calculate_score(y_test, preds)
print("The LB score is: {0}".format(score))

The LB score is: 0.7329862565768264


In [ ]:
fp = "models/lstm_{0}_SCORE_{1}".format(datetime.now().strftime("%Y-%m-%d %H:%M:%S"), np.round(score,5)) 
model.save(fp)

In [ ]:
model.summary()